In [ ]:
import os
if os.path.isdir('../../UnifiedOntology-Pipeline'): os.chdir('../')

In [ ]:
import import_ipynb
from tool.extractClass import *
from pipeline.connectIndividuals import getNamespace, clear_serialize

In [ ]:
loginHuggingFace()

In [ ]:
path_to_onto = "./pipeline/3_mergeOutput/UnifiedOntology.ttl"
df = getOntoClasses(path_to_onto)

In [ ]:
name_tokenizer = "bert-base-uncased"
name_model = "OhWayTee/bert-taxonomy"
name_pipeline = "text-classification"

In [ ]:
classifier = getClassifier(name_tokenizer, name_model, name_pipeline)

In [ ]:
df = df[:10]

In [ ]:
df_pred = getPredictions(classifier, df)

In [ ]:
df_top = sortPredictions(df_pred)

In [ ]:
df_top

In [ ]:
df_superclass = df_top[df_top['label'] == "LABEL_1"] # classB is SUBCLASS of classA
df_subclass = df_top[df_top['label'] == "LABEL_2"] # classA is SUBCLASS of classB

In [ ]:
df_superclass

In [ ]:
df_subclass

In [ ]:
g = Graph()
g.parse(path_to_onto, format='turtle')
prefix, uri = getNamespace(g)
print(prefix, uri)

In [ ]:
for idx, row in df_superclass.iterrows():
    print(str(row['classA']), str(row['classB']))

In [ ]:
for idx, row in df_superclass.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
        ?subclass rdfs:label "{row['classB']}" .
        ?superclass rdfs:label "{row['classA']}" .
    }}
    """
    g.update(query)

In [ ]:
for idx, row in df_subclass.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
        ?subclass rdfs:label "{row['classA']}" .
        ?superclass rdfs:label "{row['classB']}" .
    }}
    """
    g.update(query)

In [ ]:
output_path = "./refine/4_refineOutput/RefinedUnifiedOntology.ttl"

clear_serialize(g, output_path)